In [ ]:
import torch 
from torch_geometric.nn import GCNConv, SAGEConv
from torch_geometric.data import Data
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as spio